#  For EdgeDrive, use x64 not ARM64c

In [ ]:
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import time

from IPython.display import clear_output

In [ ]:
# Step 1: Read the Excel file into a DataFrame
#file_path = "dummy.xlsx"
file_path = "GAE PCC Summary 2025 - Copy.xlsx"
df = pd.read_excel(file_path)

# Step 2: Remove duplicate rows based on the column "Workflow Backstop ID"
df = df.drop_duplicates(subset=["Workflow Backstop ID"])
print(df.shape)

In [ ]:
# Step 3: Create the baseline folder "Download" if it doesn't exist
base_folder = r"C:\Users\A0770554\PCC2025"
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

In [ ]:
# Function to setup and return Edge WebDriver with specified download directory
def setup_driver(download_directory):
    edge_options = Options()
    edge_options.add_argument("--start-maximized")
#    edge_options.add_argument(r"user-data-dir=C:\Users\A0770554\AppData\Local\Microsoft\Edge\User Data")
#    edge_options.add_argument("profile-directory=Profile 1")

    prefs = {
        "download.default_directory": download_directory + r"\\",
        "savefile.default_directory": download_directory + r"\\",
#        'download.prompt_for_download': False,
#        'download.directory_upgrade': True,
#        'safebrowsing.enabled': True
    }
    edge_options.add_experimental_option('prefs', prefs)
    
    # Replace with the path to your Edge WebDriver
    webdriver_path = "msedgedriver.exe"
    service = Service(webdriver_path)

    driver = webdriver.Edge(service=service, options=edge_options)
    return driver

In [ ]:
# Create new dataframe
new_df = pd.DataFrame(columns=["Number", "Workflow ID", "Title", "Download Success"])

counter = 1

# Step 4: Loop through the DataFrame and set the download directory dynamically
for index, row in df.iterrows():

    print(counter)

    title = row['Title']
    workflow_id = row['Workflow Backstop ID']

    # Remove special characters from the title
    sanitized_title = re.sub(r'[<>:"/\\|?*]', '', title)
    
    # Ensure the download directory path does not exceed 250 characters
    download_directory = os.path.join(base_folder, str(workflow_id) + " " + sanitized_title)

    if len(download_directory) > 250:
        truncated_title = sanitized_title[:250 - len(base_folder) - 1]
        download_directory = os.path.join(base_folder, truncated_title)
    
    # Create the download directory if it doesn't exist
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    print(download_directory)
    driver = setup_driver(download_directory)
    
    # Step 5: Navigate to the specified URL
    url = f"https://inview.backstopsolutions.com/backstop/workflow/WorkflowProcess.action?documents=&id={workflow_id}"
    driver.get(url)

    time.sleep(32) #need to sign in to website

    # Add a new row to the new DataFrame
    new_row = {"Number": counter, "Workflow ID": workflow_id, "Title": title, "Download Success": "No"}
    new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

    # Find all download links dynamically
    try:
        download_links = driver.find_elements(By.XPATH, "//a[contains(text(), 'Download')]")
        print(download_links)
        if download_links:
            print("Found download links:")
            for link in download_links:
                link.click()
                href = link.get_attribute("href")
                print(href)
                # Update the "Download Success" column for the corresponding row
                new_df.loc[new_df['Number'] == counter, 'Download Success'] = "Yes"
                
        else:
            print("No download links found.")
            new_df.loc[new_df['Number'] == counter, 'Download Success'] = "No"
            #driver.quit()
            #sys.exit("No download links found. Stopping the script.")

    except Exception as e:
        print("Error finding download links:", e)

    time.sleep(10) #Let the downloads complete
    
    # Close the browser
    driver.quit()

    counter = counter + 1

    # Clear the notebook output 
    clear_output(wait=True) 



In [ ]:
# Export the new DataFrame to an Excel file named "Scrape Checklist"
new_df.to_excel("Scrape Checklist.xlsx", index=False)

### Check which workflow folder does not have an excel downloaded

In [ ]:
# List to store folder names without Excel files
folders_without_excel = []

# Loop through each subfolder in the base folder
for root, dirs, files in os.walk(base_folder):
    for dir_name in dirs:
        folder_path = os.path.join(root, dir_name)
        # Check if there are any Excel files in the current folder
        if not any(file.endswith(('.xls', '.xlsx', '.xlsm')) for file in os.listdir(folder_path)):
            folders_without_excel.append(folder_path)

# Create a pandas DataFrame from the list of folder names
df = pd.DataFrame(folders_without_excel, columns=['Folder'])

# Export the DataFrame to a CSV file
output_file = os.path.join(base_folder, "FoldersWithoutExcel.csv")
df.to_csv(output_file, index=False)

print(f"CSV file 'FoldersWithoutExcel.csv' has been created at {output_file}")